In [28]:
import random

random.choice(list((1,2,3,4)))

1

In [38]:
import numpy

numpy.sum(numpy.random.randint(1,7,(100,10)), axis=0)

array([329, 329, 350, 352, 361, 383, 355, 324, 393, 375])

In [40]:
import time 

start_time = time.clock()
stop_time = time.clock()
stop_time - start_time

AttributeError: module 'time' has no attribute 'clock'